# Chapter 15: State Management

Managing state effectively is one of the most critical aspects of building scalable Next.js applications. With the introduction of Server Components in the App Router, state management has evolved beyond traditional client-side patterns to encompass server-side state, streaming data, and seamless synchronization between server and client boundaries.

By the end of this chapter, you'll master React Context API in the App Router era, server-side state patterns, modern client-side libraries like Zustand and Redux, URL state management, form handling, optimistic updates, and strategies for keeping server and client state synchronized.

## 15.1 React Context API

React Context provides a way to pass data through the component tree without prop drilling, but its usage has evolved with Server Components.

### Context in Client Components

Context requires the `"use client"` directive and works within client component boundaries:

```typescript
// components/theme-provider.tsx
'use client';

import { createContext, useContext, useEffect, useState } from 'react';

type Theme = 'light' | 'dark' | 'system';

interface ThemeContextType {
  theme: Theme;
  setTheme: (theme: Theme) => void;
  resolvedTheme: 'light' | 'dark';
}

const ThemeContext = createContext<ThemeContextType | undefined>(undefined);

export function ThemeProvider({ children }: { children: React.ReactNode }) {
  const [theme, setTheme] = useState<Theme>('system');
  const [resolvedTheme, setResolvedTheme] = useState<'light' | 'dark'>('light');

  useEffect(() => {
    const root = window.document.documentElement;
    const systemTheme = window.matchMedia('(prefers-color-scheme: dark)').matches ? 'dark' : 'light';
    const resolved = theme === 'system' ? systemTheme : theme;
    
    setResolvedTheme(resolved);
    root.classList.remove('light', 'dark');
    root.classList.add(resolved);
  }, [theme]);

  return (
    <ThemeContext.Provider value={{ theme, setTheme, resolvedTheme }}>
      {children}
    </ThemeContext.Provider>
  );
}

export const useTheme = () => {
  const context = useContext(ThemeContext);
  if (!context) throw new Error('useTheme must be used within ThemeProvider');
  return context;
};
```

```typescript
// app/layout.tsx
import { ThemeProvider } from '@/components/theme-provider';

export default function RootLayout({ children }: { children: React.ReactNode }) {
  return (
    <html lang="en" suppressHydrationWarning>
      <body>
        <ThemeProvider>{children}</ThemeProvider>
      </body>
    </html>
  );
}
```

### Server Context Limitations

Server Components cannot use React Context directly, but you can pass data as props:

```typescript
// app/page.tsx (Server Component)
import { UserProvider } from '@/components/user-provider';

async function getUser() {
  const res = await fetch('https://api.example.com/user', {
    cache: 'no-store',
  });
  return res.json();
}

export default async function Page() {
  const user = await getUser();
  
  // Pass server data to client context via props
  return (
    <UserProvider initialUser={user}>
      <Dashboard />
    </UserProvider>
  );
}

// components/user-provider.tsx
'use client';

import { createContext, useContext, useState } from 'react';

const UserContext = createContext(null);

export function UserProvider({ 
  children, 
  initialUser 
}: { 
  children: React.ReactNode; 
  initialUser: any;
}) {
  const [user, setUser] = useState(initialUser);
  
  return (
    <UserContext.Provider value={{ user, setUser }}>
      {children}
    </UserContext.Provider>
  );
}
```

## 15.2 Server-Side State Patterns

Server Components can fetch and manage state directly, reducing client-side JavaScript.

### Direct Data Fetching

Fetch data where it's needed without prop drilling:

```typescript
// app/dashboard/page.tsx
import { getServerSession } from '@/lib/auth';
import { db } from '@/lib/db';

// Fetch user-specific data on the server
async function getDashboardData(userId: string) {
  const [user, recentActivity, stats] = await Promise.all([
    db.user.findUnique({ where: { id: userId } }),
    db.activity.findMany({
      where: { userId },
      orderBy: { createdAt: 'desc' },
      take: 5,
    }),
    db.stats.aggregate({
      where: { userId },
      _sum: { views: true, clicks: true },
    }),
  ]);
  
  return { user, recentActivity, stats };
}

export default async function DashboardPage() {
  const session = await getServerSession();
  
  if (!session) {
    redirect('/login');
  }
  
  const data = await getDashboardData(session.userId);
  
  return (
    <div className="container mx-auto py-8">
      <h1 className="text-3xl font-bold mb-6">
        Welcome, {data.user?.name}
      </h1>
      
      <div className="grid grid-cols-1 md:grid-cols-3 gap-6 mb-8">
        <StatCard 
          title="Total Views" 
          value={data.stats._sum.views || 0} 
        />
        <StatCard 
          title="Total Clicks" 
          value={data.stats._sum.clicks || 0} 
        />
      </div>
      
      <RecentActivity items={data.recentActivity} />
    </div>
  );
}
```

### Caching Server State

Control how server data is cached and revalidated:

```typescript
// lib/data.ts
import { unstable_cache } from 'next/cache';

// Cached function for expensive operations
export const getCachedStats = unstable_cache(
  async (category: string) => {
    // Expensive database query
    const stats = await db.stats.aggregate({
      where: { category },
      _count: true,
      _avg: { value: true },
    });
    
    return stats;
  },
  ['stats'], // Cache key
  { revalidate: 3600 } // 1 hour
);

// Different caching strategies
export async function getRealtimeData() {
  // No caching - fresh data every time
  const res = await fetch('https://api.example.com/live', {
    cache: 'no-store',
  });
  return res.json();
}

export async function getHourlyData() {
  // Revalidate every hour
  const res = await fetch('https://api.example.com/hourly', {
    next: { revalidate: 3600 },
  });
  return res.json();
}

export async function getDailyData() {
  // Tag-based revalidation
  const res = await fetch('https://api.example.com/daily', {
    next: { tags: ['daily-data'] },
  });
  return res.json();
}
```

### Server State Composition

Compose multiple data sources in server components:

```typescript
// app/composed/page.tsx
import { Suspense } from 'react';
import { unstable_noStore } from 'next/cache';

// Parallel data fetching with suspense boundaries
async function WeatherWidget() {
  unstable_noStore(); // Dynamic data
  
  const weather = await fetch('https://api.weather.com/current', {
    cache: 'no-store',
  }).then(r => r.json());
  
  return (
    <div className="p-4 bg-blue-50 rounded">
      <h3>Current Weather</h3>
      <p>{weather.temperature}°C - {weather.condition}</p>
    </div>
  );
}

async function StockTicker() {
  unstable_noStore();
  
  const stocks = await fetch('https://api.stocks.com/market', {
    cache: 'no-store',
  }).then(r => r.json());
  
  return (
    <div className="p-4 bg-green-50 rounded">
      <h3>Market Summary</h3>
      <p>S&P 500: {stocks.sp500}</p>
    </div>
  );
}

// Cached component
async function Announcements() {
  const announcements = await fetch('https://api.cms.com/announcements', {
    next: { revalidate: 300 }, // 5 minutes
  }).then(r => r.json());
  
  return (
    <div className="p-4 bg-yellow-50 rounded">
      <h3>Announcements</h3>
      <ul>
        {announcements.map((a: any) => (
          <li key={a.id}>{a.title}</li>
        ))}
      </ul>
    </div>
  );
}

export default function ComposedPage() {
  return (
    <div className="container mx-auto py-8">
      <h1 className="text-3xl font-bold mb-8">Dashboard</h1>
      
      <div className="grid grid-cols-1 md:grid-cols-3 gap-6">
        {/* Static/cached content loads immediately */}
        <Announcements />
        
        {/* Dynamic content streams in */}
        <Suspense fallback={<div className="p-4 bg-gray-100 rounded">Loading weather...</div>}>
          <WeatherWidget />
        </Suspense>
        
        <Suspense fallback={<div className="p-4 bg-gray-100 rounded">Loading stocks...</div>}>
          <StockTicker />
        </Suspense>
      </div>
    </div>
  );
}
```

## 15.3 Client-Side State Libraries (Zustand, Redux)

For complex client-side state, modern libraries offer better performance and developer experience than Context.

### Zustand Integration

Zustand is lightweight, unopinionated, and works seamlessly with Next.js:

```typescript
// stores/counter-store.ts
'use client';

import { create } from 'zustand';
import { persist } from 'zustand/middleware';

interface CounterState {
  count: number;
  increment: () => void;
  decrement: () => void;
  reset: () => void;
}

export const useCounterStore = create<CounterState>()(
  persist(
    (set) => ({
      count: 0,
      increment: () => set((state) => ({ count: state.count + 1 })),
      decrement: () => set((state) => ({ count: state.count - 1 })),
      reset: () => set({ count: 0 }),
    }),
    {
      name: 'counter-storage',
    }
  )
);

// stores/user-store.ts
import { create } from 'zustand';

interface User {
  id: string;
  name: string;
  email: string;
}

interface UserState {
  user: User | null;
  isLoading: boolean;
  setUser: (user: User | null) => void;
  updateProfile: (data: Partial<User>) => void;
}

export const useUserStore = create<UserState>((set) => ({
  user: null,
  isLoading: false,
  setUser: (user) => set({ user }),
  updateProfile: (data) =>
    set((state) => ({
      user: state.user ? { ...state.user, ...data } : null,
    })),
}));
```

```typescript
// components/counter.tsx
'use client';

import { useCounterStore } from '@/stores/counter-store';

export function Counter() {
  const { count, increment, decrement } = useCounterStore();
  
  return (
    <div className="flex items-center gap-4 p-4 border rounded">
      <button 
        onClick={decrement}
        className="px-4 py-2 bg-red-500 text-white rounded"
      >
        -
      </button>
      <span className="text-2xl font-bold">{count}</span>
      <button 
        onClick={increment}
        className="px-4 py-2 bg-green-500 text-white rounded"
      >
        +
      </button>
    </div>
  );
}
```

### Redux Toolkit with Next.js

For larger applications, Redux Toolkit provides structure and devtools:

```typescript
// store/index.ts
import { configureStore } from '@reduxjs/toolkit';
import cartReducer from './slices/cartSlice';
import uiReducer from './slices/uiSlice';

export const makeStore = () => {
  return configureStore({
    reducer: {
      cart: cartReducer,
      ui: uiReducer,
    },
  });
};

export type AppStore = ReturnType<typeof makeStore>;
export type RootState = ReturnType<AppStore['getState']>;
export type AppDispatch = AppStore['dispatch'];
```

```typescript
// store/provider.tsx
'use client';

import { useRef } from 'react';
import { Provider } from 'react-redux';
import { makeStore, AppStore } from './index';

export function StoreProvider({ children }: { children: React.ReactNode }) {
  const storeRef = useRef<AppStore>();
  
  if (!storeRef.current) {
    storeRef.current = makeStore();
  }
  
  return <Provider store={storeRef.current}>{children}</Provider>;
}
```

```typescript
// store/slices/cartSlice.ts
import { createSlice, PayloadAction } from '@reduxjs/toolkit';

interface CartItem {
  id: string;
  name: string;
  price: number;
  quantity: number;
}

interface CartState {
  items: CartItem[];
  total: number;
}

const initialState: CartState = {
  items: [],
  total: 0,
};

const cartSlice = createSlice({
  name: 'cart',
  initialState,
  reducers: {
    addItem: (state, action: PayloadAction<CartItem>) => {
      const existingItem = state.items.find(
        item => item.id === action.payload.id
      );
      
      if (existingItem) {
        existingItem.quantity += action.payload.quantity;
      } else {
        state.items.push(action.payload);
      }
      
      state.total = state.items.reduce(
        (sum, item) => sum + item.price * item.quantity, 
        0
      );
    },
    removeItem: (state, action: PayloadAction<string>) => {
      state.items = state.items.filter(item => item.id !== action.payload);
      state.total = state.items.reduce(
        (sum, item) => sum + item.price * item.quantity, 
        0
      );
    },
  },
});

export const { addItem, removeItem } = cartSlice.actions;
export default cartSlice.reducer;
```

## 15.4 URL State Management

Store state in the URL for shareable, bookmarkable application states.

### Search Parameters as State

Use `useSearchParams` for filter and sort state:

```typescript
// components/product-filter.tsx
'use client';

import { useSearchParams, usePathname, useRouter } from 'next/navigation';
import { useCallback } from 'react';

export function ProductFilter() {
  const router = useRouter();
  const pathname = usePathname();
  const searchParams = useSearchParams();
  
  // Get current values from URL
  const category = searchParams.get('category') || 'all';
  const sort = searchParams.get('sort') || 'newest';
  const minPrice = searchParams.get('minPrice') || '';
  
  // Create query string updater
  const createQueryString = useCallback(
    (name: string, value: string) => {
      const params = new URLSearchParams(searchParams.toString());
      
      if (value) {
        params.set(name, value);
      } else {
        params.delete(name);
      }
      
      return params.toString();
    },
    [searchParams]
  );
  
  const updateFilter = (name: string, value: string) => {
    const query = createQueryString(name, value);
    router.push(`${pathname}?${query}`, { scroll: false });
  };
  
  return (
    <div className="space-y-4 p-4 border rounded">
      <div>
        <label className="block text-sm font-medium mb-2">Category</label>
        <select
          value={category}
          onChange={(e) => updateFilter('category', e.target.value)}
          className="w-full border rounded p-2"
        >
          <option value="all">All Categories</option>
          <option value="electronics">Electronics</option>
          <option value="clothing">Clothing</option>
        </select>
      </div>
      
      <div>
        <label className="block text-sm font-medium mb-2">Sort By</label>
        <select
          value={sort}
          onChange={(e) => updateFilter('sort', e.target.value)}
          className="w-full border rounded p-2"
        >
          <option value="newest">Newest First</option>
          <option value="price-asc">Price: Low to High</option>
          <option value="price-desc">Price: High to Low</option>
        </select>
      </div>
      
      <div>
        <label className="block text-sm font-medium mb-2">Min Price</label>
        <input
          type="number"
          value={minPrice}
          onChange={(e) => updateFilter('minPrice', e.target.value)}
          className="w-full border rounded p-2"
          placeholder="0"
        />
      </div>
    </div>
  );
}
```

### Synchronized URL State Hook

Create a reusable hook for URL state:

```typescript
// hooks/use-url-state.ts
'use client';

import { useSearchParams, useRouter, usePathname } from 'next/navigation';
import { useCallback } from 'react';

type UrlState = Record<string, string | undefined>;

export function useUrlState() {
  const router = useRouter();
  const pathname = usePathname();
  const searchParams = useSearchParams();
  
  const getState = useCallback((): UrlState => {
    const state: UrlState = {};
    searchParams.forEach((value, key) => {
      state[key] = value;
    });
    return state;
  }, [searchParams]);
  
  const setState = useCallback(
    (newState: Partial<UrlState>, options?: { replace?: boolean; scroll?: boolean }) => {
      const params = new URLSearchParams(searchParams.toString());
      
      Object.entries(newState).forEach(([key, value]) => {
        if (value === undefined || value === '') {
          params.delete(key);
        } else {
          params.set(key, value);
        }
      });
      
      const query = params.toString();
      const url = query ? `${pathname}?${query}` : pathname;
      
      if (options?.replace) {
        router.replace(url, { scroll: options.scroll ?? false });
      } else {
        router.push(url, { scroll: options.scroll ?? false });
      }
    },
    [pathname, router, searchParams]
  );
  
  return { state: getState(), setState, getState };
}
```

### Complex State Serialization

Handle complex objects in URLs:

```typescript
// hooks/use-complex-url-state.ts
'use client';

import { useSearchParams, useRouter, usePathname } from 'next/navigation';
import { useCallback, useMemo } from 'react';

interface FilterState {
  categories: string[];
  priceRange: [number, number];
  tags: string[];
  inStockOnly: boolean;
}

export function useComplexFilterState() {
  const router = useRouter();
  const pathname = usePathname();
  const searchParams = useSearchParams();
  
  const filters = useMemo((): FilterState => {
    return {
      categories: searchParams.get('cat')?.split(',') || [],
      priceRange: [
        parseInt(searchParams.get('min') || '0'),
        parseInt(searchParams.get('max') || '1000'),
      ],
      tags: searchParams.get('tags')?.split(',') || [],
      inStockOnly: searchParams.get('stock') === 'true',
    };
  }, [searchParams]);
  
  const setFilters = useCallback(
    (newFilters: Partial<FilterState>) => {
      const params = new URLSearchParams(searchParams.toString());
      const merged = { ...filters, ...newFilters };
      
      // Serialize arrays
      if (merged.categories.length > 0) {
        params.set('cat', merged.categories.join(','));
      } else {
        params.delete('cat');
      }
      
      if (merged.tags.length > 0) {
        params.set('tags', merged.tags.join(','));
      } else {
        params.delete('tags');
      }
      
      // Serialize range
      params.set('min', merged.priceRange[0].toString());
      params.set('max', merged.priceRange[1].toString());
      
      // Serialize boolean
      if (merged.inStockOnly) {
        params.set('stock', 'true');
      } else {
        params.delete('stock');
      }
      
      router.push(`${pathname}?${params.toString()}`, { scroll: false });
    },
    [filters, pathname, router, searchParams]
  );
  
  return { filters, setFilters };
}
```

## 15.5 Form State Handling

Manage complex form state with validation and submission handling.

### Server Action Forms

Use Server Actions for form submission without client-side JavaScript:

```typescript
// app/contact/page.tsx
import { submitContactForm } from './actions';

export default function ContactPage() {
  return (
    <form action={submitContactForm} className="space-y-4 max-w-md">
      <div>
        <label htmlFor="name" className="block text-sm font-medium">Name</label>
        <input
          type="text"
          id="name"
          name="name"
          required
          className="w-full border rounded p-2"
        />
      </div>
      
      <div>
        <label htmlFor="email" className="block text-sm font-medium">Email</label>
        <input
          type="email"
          id="email"
          name="email"
          required
          className="w-full border rounded p-2"
        />
      </div>
      
      <div>
        <label htmlFor="message" className="block text-sm font-medium">Message</label>
        <textarea
          id="message"
          name="message"
          required
          rows={4}
          className="w-full border rounded p-2"
        />
      </div>
      
      <button 
        type="submit"
        className="w-full bg-blue-500 text-white py-2 rounded hover:bg-blue-600"
      >
        Send Message
      </button>
    </form>
  );
}

// app/contact/actions.ts
'use server';

import { revalidatePath } from 'next/cache';
import { z } from 'zod';

const contactSchema = z.object({
  name: z.string().min(2),
  email: z.string().email(),
  message: z.string().min(10),
});

export async function submitContactForm(formData: FormData) {
  const data = {
    name: formData.get('name'),
    email: formData.get('email'),
    message: formData.get('message'),
  };
  
  const result = contactSchema.safeParse(data);
  
  if (!result.success) {
    return { error: 'Invalid form data', details: result.error.errors };
  }
  
  // Save to database
  await db.contact.create({
    data: result.data,
  });
  
  revalidatePath('/contact');
  
  return { success: true };
}
```

### React Hook Form with Zod

For complex client-side forms with validation:

```typescript
// components/user-profile-form.tsx
'use client';

import { useForm } from 'react-hook-form';
import { zodResolver } from '@hookform/resolvers/zod';
import { z } from 'zod';
import { useTransition } from 'react';

const profileSchema = z.object({
  username: z.string().min(3).max(20),
  bio: z.string().max(500).optional(),
  website: z.string().url().optional().or(z.literal('')),
  isPublic: z.boolean(),
});

type ProfileFormData = z.infer<typeof profileSchema>;

export function UserProfileForm({ 
  initialData 
}: { 
  initialData: Partial<ProfileFormData> 
}) {
  const [isPending, startTransition] = useTransition();
  
  const {
    register,
    handleSubmit,
    formState: { errors, isDirty },
    reset,
  } = useForm<ProfileFormData>({
    resolver: zodResolver(profileSchema),
    defaultValues: {
      username: initialData.username || '',
      bio: initialData.bio || '',
      website: initialData.website || '',
      isPublic: initialData.isPublic ?? true,
    },
  });
  
  const onSubmit = async (data: ProfileFormData) => {
    startTransition(async () => {
      try {
        const response = await fetch('/api/profile', {
          method: 'PUT',
          headers: { 'Content-Type': 'application/json' },
          body: JSON.stringify(data),
        });
        
        if (!response.ok) throw new Error('Failed to update');
        
        // Reset form with new values to clear dirty state
        reset(data);
      } catch (error) {
        console.error('Update failed:', error);
      }
    });
  };
  
  return (
    <form onSubmit={handleSubmit(onSubmit)} className="space-y-4">
      <div>
        <label className="block text-sm font-medium">Username</label>
        <input
          {...register('username')}
          className="w-full border rounded p-2"
        />
        {errors.username && (
          <p className="text-red-500 text-sm">{errors.username.message}</p>
        )}
      </div>
      
      <div>
        <label className="block text-sm font-medium">Bio</label>
        <textarea
          {...register('bio')}
          className="w-full border rounded p-2"
          rows={4}
        />
      </div>
      
      <div>
        <label className="block text-sm font-medium">Website</label>
        <input
          {...register('website')}
          className="w-full border rounded p-2"
          placeholder="https://example.com"
        />
        {errors.website && (
          <p className="text-red-500 text-sm">{errors.website.message}</p>
        )}
      </div>
      
      <div className="flex items-center gap-2">
        <input
          type="checkbox"
          {...register('isPublic')}
          id="isPublic"
        />
        <label htmlFor="isPublic">Make profile public</label>
      </div>
      
      <button
        type="submit"
        disabled={!isDirty || isPending}
        className="bg-blue-500 text-white px-4 py-2 rounded disabled:opacity-50"
      >
        {isPending ? 'Saving...' : 'Save Changes'}
      </button>
    </form>
  );
}
```

## 15.6 Optimistic Updates

Provide immediate feedback while server operations complete.

### Optimistic UI Pattern

Update UI immediately before server confirmation:

```typescript
// components/like-button.tsx
'use client';

import { useState } from 'react';
import { useRouter } from 'next/navigation';

interface LikeButtonProps {
  postId: string;
  initialLikes: number;
  isLiked: boolean;
}

export function LikeButton({ postId, initialLikes, isLiked: initialIsLiked }: LikeButtonProps) {
  const [optimisticLikes, setOptimisticLikes] = useState(initialLikes);
  const [optimisticIsLiked, setOptimisticIsLiked] = useState(initialIsLiked);
  const [isPending, setIsPending] = useState(false);
  const router = useRouter();
  
  const handleLike = async () => {
    // Optimistic update
    const newLikedState = !optimisticIsLiked;
    setOptimisticIsLiked(newLikedState);
    setOptimisticLikes(prev => newLikedState ? prev + 1 : prev - 1);
    setIsPending(true);
    
    try {
      const response = await fetch(`/api/posts/${postId}/like`, {
        method: 'POST',
      });
      
      if (!response.ok) {
        throw new Error('Failed to update like');
      }
      
      // Refresh server components
      router.refresh();
    } catch (error) {
      // Revert optimistic update on error
      setOptimisticIsLiked(!newLikedState);
      setOptimisticLikes(prev => newLikedState ? prev - 1 : prev + 1);
      console.error('Like failed:', error);
    } finally {
      setIsPending(false);
    }
  };
  
  return (
    <button
      onClick={handleLike}
      disabled={isPending}
      className={`flex items-center gap-2 px-4 py-2 rounded ${
        optimisticIsLiked 
          ? 'bg-red-500 text-white' 
          : 'bg-gray-200 text-gray-700'
      } ${isPending ? 'opacity-70' : ''}`}
    >
      <span>{optimisticIsLiked ? '❤️' : '🤍'}</span>
      <span>{optimisticLikes}</span>
    </button>
  );
}
```

### useOptimistic Hook (React 18+)

Use the built-in `useOptimistic` hook for cleaner code:

```typescript
// components/todo-list.tsx
'use client';

import { useOptimistic, useTransition } from 'react';
import { createTodo, deleteTodo } from './actions';

interface Todo {
  id: string;
  text: string;
  completed: boolean;
}

export function TodoList({ initialTodos }: { initialTodos: Todo[] }) {
  const [isPending, startTransition] = useTransition();
  
  const [optimisticTodos, addOptimisticTodo] = useOptimistic(
    initialTodos,
    (state: Todo[], newTodo: Todo) => [...state, newTodo]
  );
  
  const handleAdd = async (formData: FormData) => {
    const text = formData.get('text') as string;
    
    const optimisticTodo: Todo = {
      id: `temp-${Date.now()}`,
      text,
      completed: false,
    };
    
    startTransition(async () => {
      // Optimistic update
      addOptimisticTodo(optimisticTodo);
      
      // Server action
      await createTodo(formData);
    });
  };
  
  return (
    <div className="space-y-4">
      <form action={handleAdd} className="flex gap-2">
        <input
          name="text"
          required
          className="flex-1 border rounded p-2"
          placeholder="Add todo..."
        />
        <button 
          type="submit"
          disabled={isPending}
          className="bg-blue-500 text-white px-4 py-2 rounded disabled:opacity-50"
        >
          Add
        </button>
      </form>
      
      <ul className="space-y-2">
        {optimisticTodos.map((todo) => (
          <li 
            key={todo.id}
            className={`flex items-center justify-between p-3 border rounded ${
              todo.id.startsWith('temp-') ? 'opacity-50' : ''
            }`}
          >
            <span className={todo.completed ? 'line-through' : ''}>
              {todo.text}
            </span>
            {!todo.id.startsWith('temp-') && (
              <form action={() => deleteTodo(todo.id)}>
                <button type="submit" className="text-red-500">
                  Delete
                </button>
              </form>
            )}
          </li>
        ))}
      </ul>
    </div>
  );
}
```

## 15.7 State Synchronization

Keep server and client state synchronized effectively.

### Hydration and Initial State

Pass server state to client components correctly:

```typescript
// app/products/page.tsx
import { ProductList } from '@/components/product-list';
import { getProducts } from '@/lib/data';

export default async function ProductsPage() {
  const products = await getProducts();
  
  return (
    <div className="container mx-auto py-8">
      <h1 className="text-3xl font-bold mb-8">Products</h1>
      
      {/* Pass server data as props to client component */}
      <ProductList initialProducts={products} />
    </div>
  );
}

// components/product-list.tsx
'use client';

import { useState } from 'react';

interface Product {
  id: string;
  name: string;
  price: number;
}

export function ProductList({ initialProducts }: { initialProducts: Product[] }) {
  // Initialize client state with server data
  const [products, setProducts] = useState(initialProducts);
  const [isRefreshing, setIsRefreshing] = useState(false);
  
  const refreshProducts = async () => {
    setIsRefreshing(true);
    try {
      const res = await fetch('/api/products');
      const newProducts = await res.json();
      setProducts(newProducts);
    } finally {
      setIsRefreshing(false);
    }
  };
  
  return (
    <div>
      <button 
        onClick={refreshProducts}
        disabled={isRefreshing}
        className="mb-4 px-4 py-2 bg-gray-200 rounded"
      >
        {isRefreshing ? 'Refreshing...' : 'Refresh'}
      </button>
      
      <div className="grid grid-cols-3 gap-4">
        {products.map(product => (
          <div key={product.id} className="border p-4 rounded">
            <h3>{product.name}</h3>
            <p>${product.price}</p>
          </div>
        ))}
      </div>
    </div>
  );
}
```

### Real-time Synchronization

Use Server-Sent Events or WebSockets for live updates:

```typescript
// hooks/use-realtime-data.ts
'use client';

import { useEffect, useState } from 'react';

export function useRealtimeData<T>(endpoint: string, initialData: T) {
  const [data, setData] = useState<T>(initialData);
  const [isConnected, setIsConnected] = useState(false);
  
  useEffect(() => {
    const eventSource = new EventSource(endpoint);
    
    eventSource.onopen = () => setIsConnected(true);
    
    eventSource.onmessage = (event) => {
      const newData = JSON.parse(event.data);
      setData(newData);
    };
    
    eventSource.onerror = () => {
      setIsConnected(false);
      eventSource.close();
    };
    
    return () => {
      eventSource.close();
    };
  }, [endpoint]);
  
  return { data, isConnected };
}

// Usage
export function LiveDashboard() {
  const { data: stats, isConnected } = useRealtimeData(
    '/api/stats/stream',
    { visitors: 0, sales: 0 }
  );
  
  return (
    <div>
      <div className={`inline-block w-3 h-3 rounded-full ${
        isConnected ? 'bg-green-500' : 'bg-red-500'
      }`} />
      <div>Visitors: {stats.visitors}</div>
      <div>Sales: {stats.sales}</div>
    </div>
  );
}
```

### SWR for Data Fetching

Use SWR for automatic caching, revalidation, and synchronization:

```typescript
// hooks/use-user.ts
'use client';

import useSWR from 'swr';

const fetcher = (url: string) => fetch(url).then(r => r.json());

export function useUser(id: string) {
  const { data, error, isLoading, mutate } = useSWR(
    `/api/users/${id}`,
    fetcher,
    {
      refreshInterval: 30000, // Revalidate every 30s
      revalidateOnFocus: true,
      revalidateOnReconnect: true,
    }
  );
  
  const updateUser = async (updates: Partial<User>) => {
    // Optimistic update
    await mutate(
      async (currentData) => {
        const res = await fetch(`/api/users/${id}`, {
          method: 'PATCH',
          headers: { 'Content-Type': 'application/json' },
          body: JSON.stringify(updates),
        });
        
        if (!res.ok) throw new Error('Update failed');
        return res.json();
      },
      {
        optimisticData: (current) => ({ ...current, ...updates }),
        rollbackOnError: true,
      }
    );
  };
  
  return {
    user: data,
    isLoading,
    isError: error,
    updateUser,
    refresh: mutate,
  };
}
```

## Key Takeaways from Chapter 15

1. **React Context API**: Context works only in Client Components (`'use client'`). Use it for global UI state like themes, authentication status, or feature flags. For Server Components, pass data as props or fetch directly in the component.

2. **Server-Side State**: Leverage Server Components to fetch data directly without client-side state management. Use `unstable_cache` for expensive operations, and compose multiple data sources with Suspense boundaries for optimal loading states.

3. **Client-Side Libraries**: Use Zustand for lightweight global state with minimal boilerplate, or Redux Toolkit for complex applications requiring strict data flow and devtools. Always wrap providers in `'use client'` components and initialize stores in `useRef` to prevent recreation on renders.

4. **URL State Management**: Store filter, sort, and pagination state in URLs using `useSearchParams` for shareable, bookmarkable application states. Create custom hooks like `useUrlState` to serialize/deserialize complex objects while keeping the browser history clean.

5. **Form State Handling**: Use Server Actions for simple forms with progressive enhancement (works without JavaScript). For complex client-side validation, use React Hook Form with Zod resolvers to manage form state, validation, and submission handling efficiently.

6. **Optimistic Updates**: Provide immediate UI feedback using `useOptimistic` or manual state management before server confirmation. Always implement error handling to revert changes if the server operation fails, and use `router.refresh()` to synchronize Server Components after mutations.

7. **State Synchronization**: Pass initial server data to Client Components via props to avoid hydration mismatches. Use SWR or React Query for client-side caching with automatic revalidation, and implement Server-Sent Events or WebSockets for real-time data synchronization across clients.

## Coming Up Next

**Chapter 16: Authentication & Authorization**

Now that you understand state management patterns, it's time to secure your application. In Chapter 16, we'll explore authentication fundamentals, NextAuth.js integration, OAuth providers, credential authentication, session management, protected routes, and role-based access control. You'll learn how to implement secure, production-ready authentication flows that work seamlessly with both Server and Client Components.